# 将覆盖率转化为自然与语言

In [2]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
import jsonlines
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time
import re
from tqdm import tqdm

## 处理语料

In [17]:
print("{\n    \"instance\": {\n        \"放火（Arson）\": [\n            “在居民楼内点燃易燃液体引发火灾”,\n            “为骗取保险金焚烧自家商铺”,\n            “在森林保护区故意丢弃未熄灭烟头引发山火”,\n            “因报复社会在公交车上泼洒汽油点火”,\n            “为销毁犯罪证据烧毁仓库”,\n            “精神疾病患者在幼儿园纵火”,\n            “恐怖分子在火车站引爆炸弹引发连环燃烧”,\n            “农民焚烧秸秆导致火势失控蔓延至周边村庄”,\n            “黑社会成员为威胁对手焚烧其车辆”,\n            “利用电子设备远程引燃化工厂原料罐”\n        ],\n        \"决水（Flooding）\": [\n            “挖开防洪堤导致下游村庄被淹”,\n            “为争夺水源炸毁邻村灌溉水渠”,\n            “在暴雨期间故意破坏城市排水系统”,\n            “非法开启水库闸门致下游农田损毁”,\n            “向地铁隧道内灌注大量污水”,\n            “为掩盖排污行为凿穿污水处理池”,\n            “盗采河沙导致河堤结构性崩塌”,\n            “故意堵塞城市河道引发内涝”,\n            “在水利枢纽控制室篡改程序引发泄洪”,\n            “为阻止追捕破坏水坝闸门制造洪水屏障”\n        ],\n        \"爆炸（Explosive）\": [\n            “在政府大楼前引爆汽车炸弹”,\n            “将自制炸药包裹放置于商场垃圾桶”,\n            “矿场工人违规使用过量炸药引发矿难”,\n            “在天然气管道关键节点安装定时炸弹”,\n            “向婚礼现场邮寄伪装礼盒的爆炸装置”,\n            “使用无人机空投爆炸物袭击电力设施”,\n            “在化工厂反应釜内混入不稳定化学品”,\n            “恐怖组织用人体炸弹袭击宗教场所”,\n            “私藏军用级炸药意外引爆居民楼”,\n            “为勒索企业威胁引爆写字楼承重柱”\n        ],\n        \"投放危险物质（DangerousSubstance）\": [\n            “向小区直饮水系统注入氰化物”,\n            “在超市熟食区喷洒肉毒杆菌培养液”,\n            “将放射性同位素混入办公楼空调系统”,\n            “邮寄炭疽病毒粉末至政府部门”,\n            “在养殖场饲料中添加重金属化合物”,\n            “向河流排放剧毒工业废料”,\n            “在幼儿园玩具表面涂抹HIV污染血液”,\n            “使用无人机播撒转基因有害花粉”,\n            “将神经毒剂装入喷雾器在地铁释放”,\n            “向食品加工厂原料库投放变质病原体”\n        ],\n        \"以危险方法危害公共安全（DangerousMethod）\": [\n            “驾驶卡车高速冲撞步行街人群”,\n            “在高铁轨道上铺设金属障碍物”,\n            “伪造空难预警引发机场大规模混乱”,\n            “使用大功率激光笔照射民航客机驾驶舱”,\n            “故意制造虚假核泄漏警报导致全城恐慌”,\n            “侵入城市交通信号系统制造连环车祸”,\n            “在高速公路撒布三角钉致多车爆胎”,\n            “释放实验室基因编辑病毒进行生物实验”,\n            “破坏核电站冷却系统引发堆芯过热”,\n            “劫持无人机编队撞击摩天大楼玻璃幕墙”\n        ],\n        \"造成严重后果（SeriousConsequences）\": [\n            “火灾导致10人以上窒息死亡”,\n            “决堤造成万亩良田永久盐碱化”,\n            “爆炸致使文化遗产建筑完全损毁”,\n            “投毒事件引发跨省传染病大流行”,\n            “危险方法造成证券交易所系统瘫痪”,\n            “公共设施破坏导致城市72小时停水停电”,\n            “放射性物质泄漏致使方圆5公里成无人区”,\n            “恶性事件引发国际外交危机”,\n            “直接经济损失超过50亿元人民币”,\n            “造成200人以上重伤需长期治疗”\n        ],\n        \"有期徒刑（FixedTermImprisonment）\": [\n            “判处有期徒刑6年”,\n            “判处有期徒刑3年半”,\n            “判处10年”\n        ]\n    }\n}")

{
    "instance": {
        "放火（Arson）": [
            “在居民楼内点燃易燃液体引发火灾”,
            “为骗取保险金焚烧自家商铺”,
            “在森林保护区故意丢弃未熄灭烟头引发山火”,
            “因报复社会在公交车上泼洒汽油点火”,
            “为销毁犯罪证据烧毁仓库”,
            “精神疾病患者在幼儿园纵火”,
            “恐怖分子在火车站引爆炸弹引发连环燃烧”,
            “农民焚烧秸秆导致火势失控蔓延至周边村庄”,
            “黑社会成员为威胁对手焚烧其车辆”,
            “利用电子设备远程引燃化工厂原料罐”
        ],
        "决水（Flooding）": [
            “挖开防洪堤导致下游村庄被淹”,
            “为争夺水源炸毁邻村灌溉水渠”,
            “在暴雨期间故意破坏城市排水系统”,
            “非法开启水库闸门致下游农田损毁”,
            “向地铁隧道内灌注大量污水”,
            “为掩盖排污行为凿穿污水处理池”,
            “盗采河沙导致河堤结构性崩塌”,
            “故意堵塞城市河道引发内涝”,
            “在水利枢纽控制室篡改程序引发泄洪”,
            “为阻止追捕破坏水坝闸门制造洪水屏障”
        ],
        "爆炸（Explosive）": [
            “在政府大楼前引爆汽车炸弹”,
            “将自制炸药包裹放置于商场垃圾桶”,
            “矿场工人违规使用过量炸药引发矿难”,
            “在天然气管道关键节点安装定时炸弹”,
            “向婚礼现场邮寄伪装礼盒的爆炸装置”,
            “使用无人机空投爆炸物袭击电力设施”,
            “在化工厂反应釜内混入不稳定化学品”,
            “恐怖组织用人体炸弹袭击宗教场所”,
            “私藏军用级炸药意

In [16]:
dataset = []
count = 0


def predicates_extract(text):
    # 使用正则表达式匹配 JSON 格式的内容
    json_pattern = r'\{.*?\}'  # 匹配从 { 开始到 } 结束的内容
    matches = re.findall(json_pattern, text, re.DOTALL)

    # 遍历匹配到的内容，尝试解析为 JSON
    for match in matches:
        try:
            data = json.loads(match)
            print("成功匹配到 JSON 格式的内容：")
            return json.dumps(data)
        except json.JSONDecodeError:
            continue  # 如果解析失败，继续尝试下一个匹配


with open('data/法律/law100_logic2coverage_v1.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        # 去掉行尾的换行符
        line = line.strip()
        # 解析 JSON 数据
        data = json.loads(line)

        try:
            logic = json.loads(data["logic"])
            count += 1

            predicates_alternative = predicates_extract(data["predicates"])

            # print(predicates_alternative)

            format = {
                "id": data["id"],
                "number": data["number"],
                "rule": logic["rule"],
                "logic_expression": logic["logicexpression"],
                "predicates_alternative": predicates_alternative
            }
        except json.JSONDecodeError:
            continue

        with jsonlines.open('data/法律/law100_logic2coverage_v2.jsonl', mode='a') as writer:
            writer.write(format)

        if(count >= 100):
            break

## 转化

In [ ]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"

# 缓存ID，假设从创建缓存的响应中获取
cache_id = "cache-ezjtmpuoc6di11gdsdm1"

# 发送DELETE请求删除缓存
response = requests.delete(
    url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
    headers={
        "Authorization": f"Bearer {api_key}"
    }
)

# 打印响应状态码
print(response.status_code)

# 如果需要，也可以打印响应内容
print(response.text)

In [ ]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"


# 读取文本文件内容
def load_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


# 创建缓存
def create_cache(file_content):
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/caching",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    cache_response = json.loads(response.text)
    return cache_response['id']


# 检查缓存是否存在且未过期
def check_cache(cache_id):
    response = requests.get(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}"
        }
    )
    if response.status_code == 200:
        return True
    return False


# 重新加载数据并更新缓存
def reload_and_update_cache(file_path, cache_id):
    new_file_content = load_file_content(file_path)
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": new_file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.put(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.status_code == 200


# 使用缓存内容并添加问题
def use_cache_with_question(cache_id, question):
    data = {
        "model": "moonshot-v1-32k",
        "messages": [
            {
                "role": "cache",
                "content": f"cache_id={cache_id};reset_ttl=3600",
            },
            {
                "role": "user",
                "content": question
            }
        ],
        "max_tokens": 8192,
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.json()['choices'][0]['message']['content']

In [ ]:
# 主流程
file_path = 'prompts/DataFountain/谓词取值.txt'
file_content = load_file_content(file_path)
print(f"File{file_path} success read")
cache_id = create_cache(file_content)
print(f"cache id:{cache_id}")
print("cache cuccess")

# # 模拟缓存过期
# time.sleep(3600)

# 检查缓存是否存在且未过期
if not check_cache(cache_id):
    print("检查不到缓存")
    # 重新加载数据并更新缓存
    if reload_and_update_cache(file_path, cache_id):
        print("缓存更新成功")
    else:
        print("缓存更新失败")

print("缓存已存在")

with open('data/法律/law100_nl2logic_v1.json', 'r', encoding='utf-8') as file:
    answer = json.load(file)

for idx, value in tqdm(enumerate(answer), total=len(answer), desc="Processing"):
    # for num, context in enumerate(answer[idx]["reference"]):
    # 使用缓存内容并添加问题
    context = value["judgement"]
    question = f"请回答关于文件内容的问题，其中[[CONTEXT]]代表的数据为{context}"
    response = use_cache_with_question(cache_id, question)
    judgement = {
        "id": value["id"],
        "number": value["number"],
        "logic": value["judgement"],
        "predicates": response
    }
    # 打开文件以进行写入，如果文件不存在，会创建文件
    with jsonlines.open('data/法律/law100_logic2coverage_v1.jsonl', mode='a') as writer:
        writer.write(judgement)
